
# Lab-05 — Agentic AI with CrewAI (via OpenRouter)
**AI Demystified: Decoding Models, Compute, and Connectivity**

**Idea.** We'll wire a tiny 2‑agent crew (**Researcher → Writer**) and call a chat model through **OpenRouter's OpenAI‑compatible API**.  
Flow: **set API key → pick model → define agents & task → run → print brief.**

> You'll need an **OpenRouter API key**. Keep prompts short; free tiers have rate limits.


## 1) Setup

In [ ]:
!pip -q install -U crewai


In [ ]:
import os

In [ ]:
BASE_URL = "https://openrouter.ai/api/v1"

In [ ]:
print(BASE_URL)

## 2) Enter your OpenRouter API key (won’t be printed)

In [ ]:

from getpass import getpass
API_KEY = getpass("Paste OpenRouter API key: ").strip()


In [ ]:

print("KEY SET" if len(API_KEY) > 0 else "NO KEY")


## 3) Export environment variables (for OpenAI‑compatible clients)

In [ ]:

os.environ["OPENAI_API_KEY"] = API_KEY
os.environ["OPENAI_API_BASE"] = BASE_URL
os.environ["OPENAI_BASE_URL"] = BASE_URL
os.environ["OPENROUTER_API_KEY"] = API_KEY
print("ENV READY")


In [ ]:
import requests
def pick_openrouter_model(preferred=None):
    preferred = preferred or [
        "mistralai/mistral-7b-instruct:free",
        "google/gemma-2-9b-it:free",
        "meta-llama/llama-3.1-8b-instruct:free",
    ]
    r = requests.get(f"{BASE_URL}/models",
                     headers={"Authorization": f"Bearer {API_KEY}"},
                     timeout=20)
    r.raise_for_status()
    ids = [m["id"] for m in r.json().get("data", [])]
    for p in preferred:
        if p in ids: return p
    free = [i for i in ids if ":free" in i]
    return free[0] if free else ids[0]

MODEL_ID = pick_openrouter_model()
print("Using MODEL_ID:", MODEL_ID)


## 4) Pick a lightweight model id

In [ ]:

# You can change this to another OpenRouter model id if you like.
MODEL_ID = "mistralai/mistral-7b-instruct:free"


In [ ]:
print(MODEL_ID)

## 5) Create the LLM handle for CrewAI

In [ ]:
from crewai import LLM
llm = LLM(
    model=MODEL_ID,
    api_key=API_KEY,
    api_base=BASE_URL,            # OpenRouter base
    custom_llm_provider="openai", # <- important
    temperature=0.0,
    max_tokens=180,
    request_timeout=30,
)
print("LLM ready:", MODEL_ID)

In [ ]:

print(type(llm).__name__)


## 6) Define two tiny agents

In [ ]:

from crewai import Agent

researcher = Agent(
    name="Researcher",
    role="Topic researcher",
    goal=("Return EXACTLY 3 bullets, 6–12 words each, STRICTLY about {topic}. "
          "No headings, no preamble, no extra lines."),
    backstory="Focused analyst who only outputs the required bullets.",
    llm=llm, allow_delegation=False, verbose=True,
)

In [ ]:

writer = Agent(
    name="Writer",
    role="Brief writer",
    goal=("Write ONE paragraph (110–140 words) strictly about {topic}, "
          "using ONLY the Researcher’s bullets. No headings, no lists."),
    backstory="Concise pre-sales writer who follows constraints.",
    llm=llm, allow_delegation=False, verbose=True,
)

In [ ]:
print(researcher.role)

In [ ]:
print(writer.role)

## 7) Define a single task and run the crew

In [ ]:
from crewai import Task, Crew, Process

topic = "What about AGI?"  # change as needed

t1 = Task(
    description=(
        f"TOPIC: {topic}\n"
        "OUTPUT EXACTLY:\n"
        "- bullet 1\n- bullet 2\n- bullet 3\n"
        "Each bullet 6–12 words. No extra text."
    ),
    expected_output="- bullet 1\n- bullet 2\n- bullet 3",
    agent=researcher,
)

t2 = Task(
    description=(
        f"Use ONLY the three bullets from the Researcher to write ONE paragraph (110–140 words) "
        f"about {topic}. No lists, no headings, no meta text."
    ),
    expected_output="One paragraph, 110–140 words.",
    agent=writer,
    context=[t1],
)

crew = Crew(agents=[researcher, writer], tasks=[t1, t2], process=Process.sequential)
print("Topic:", topic)


In [ ]:
print(topic)

## 8) Kick off the crew

In [ ]:

if not API_KEY:
    result = "No API key provided. Please rerun Section 2 with your OpenRouter key."
else:
    result = crew.kickoff(inputs={"topic": topic})


In [ ]:
print(type(result).__name__)

In [ ]:
print(str(result)[:300] + ('...' if len(str(result))>300 else ''))


## 9) Mini‑Exercises — YOU type

- Change `topic` (e.g., "customer churn signals in telecom") and re‑run Sections 7–8.
- Swap roles: let the Writer draft first, then ask Researcher to fact‑check in 3 bullets.
- Reduce verbosity (set `verbose=False`) and compare outputs.

*(One print per cell.)*


In [ ]:
# your changes here (e.g., set a new topic then re-run)


### Notes
- If you hit rate limits on free tiers, wait a moment and retry, or keep prompts short.
- To use a different model, update `MODEL_ID` (Section 4).
- When saving to GitHub from Colab, uncheck **Include the output** so your key/outputs aren’t stored.
